# Covid-19 Mask Mandates
## View and compare USA vs Global
## View and compare infection rates by US States with mandate



# Data Sources: data.gov and ourworldindata.org 

 https://catalog.data.gov/dataset/u-s-state-and-territorial-public-mask-mandates-from-april-10-2020-through-january-10-2021--e0ce3

https://catalog.data.gov/dataset/u-s-state-and-territorial-public-mask-mandates-from-april-10-2020-through-january-10-2021--e0ce3

 https://ourworldindata.org/covid-face-coverings

# Steps
1. Get mask mandate data for US country
2. Dates should be from May2020 to Dec2021
3. Group mask mandates by US states by month

In [1]:
# Dependencies
import csv
import os
import numpy as np
import requests
import pandas as pd
import gmaps
import matplotlib.pyplot as plt
import datetime
import time



In [2]:
# Study data files
file_to_load = "Resources/face-covering-policies-covid.csv"

# Read the coverings data and study results
face_coverings_df = pd.read_csv(file_to_load)
# face_coverings_df


In [3]:
# Get US data only

us_face_coverings_df = face_coverings_df.loc[face_coverings_df["Entity"] == "United States", :]
us_face_coverings_df

,Entity,Code,Day,facial_coverings
132258,United States,USA,1/1/2020,0
132259,United States,USA,1/2/2020,0
132260,United States,USA,1/3/2020,0
132261,United States,USA,1/4/2020,0
132262,United States,USA,1/5/2020,0
...,...,...,...,...
133016,United States,USA,1/28/2022,2
133017,United States,USA,1/29/2022,2
133018,United States,USA,1/30/2022,2
133019,United States,USA,1/31/2022,2


In [4]:
# Find data types

us_face_coverings_df.dtypes

Entity              object
Code                object
Day                 object
facial_coverings     int64
dtype: object

In [5]:
# Format Date column

#us_face_coverings_df["Day"] = pd.to_datetime(us_face_coverings_df["Day"], format ='%Y%m%d')

#countries_6_variants_df['date'] = pd.to_datetime(countries_6_variants_df.date)


In [10]:
# Specify date range of 01/01/2020 to 12/31/2021

begin_date = "1/1/2020"
end_date = "12/31/2021"

new_maskdata_df = us_face_coverings_df.loc[(us_face_coverings_df['Day'] >=begin_date) & (us_face_coverings_df['Day'] >=end_date)]
new_maskdata_df.head()
                                            

,Entity,Code,Day,facial_coverings
132289,United States,USA,2/1/2020,0
132290,United States,USA,2/2/2020,0
132291,United States,USA,2/3/2020,0
132292,United States,USA,2/4/2020,0
132293,United States,USA,2/5/2020,0
